## **s1reader + ISCE3 + dolphin**

In [ ]:
import os
import re
import glob

# Input workspace and meta data location
WORKSPACE = "/mnt/e/InSAR/Stuttgart/ISCE3"          # The workspace prepared for ISCE2 processing
RAW_LOCATION = "/mnt/e/InSAR/Stuttgart/RAW"         # The location of Sentinel-1 TOPS data
LOCAL_DEM_DIR = "/mnt/e/DEM/SRTMGL1/"               # The location which you stored all the local SRTM GL1 DEM tiles

# Then automatically set up the orbit and auxliary calibration files directory
ORBIT_DIR = os.path.join(WORKSPACE, "ORB")
os.makedirs(ORBIT_DIR, exist_ok=True)

The following steps: Download orbit data -> Download and stitch dem -> Generate DEM config -> Run stack processing

In [ ]:
# Locate all SAFE files in the directory
safe_files = glob.glob(os.path.join(RAW_LOCATION, 'S1*_IW_SLC*'))
if not safe_files:
    print(f"No SAFE files found in directory: {RAW_LOCATION}")

print(f"Found {len(safe_files)} SAFE files")

In [ ]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def _get_orbit_list() -> list[str]:
    """
    从 ASF（Alaska Satellite Facility）服务器获取 Sentinel-1 卫星轨道文件列表。
    
    该函数从 ASF 的轨道文件存储库获取所有可用的精密轨道文件（.EOF 格式）列表。
    这些轨道文件包含卫星的精确位置信息，用于 InSAR 数据处理中的几何校正和配准。
    
    Returns:
        list[str]: 轨道文件名列表（.EOF 格式）。如果获取失败，返回空列表。
    """
    url_root = "https://s1qc.asf.alaska.edu/aux_poeorb"

    try:
        session = _create_session()
        response = session.get(url_root, timeout=30)
        response.raise_for_status()

        # print(response.text)
        # Extract all .EOF files from the HTML
        orbit_files = re.findall(r'href="([^"]*\.EOF)"', response.text)
        print(f"Found {len(orbit_files)} orbit files.")
        return orbit_files

    except Exception as e:
        print(f"Failed to get orbit list: {e}")
        return []

def _create_session() -> requests.Session:
    """
    创建一个带有重试策略的 requests 会话。
    
    配置了自动重试机制，用于处理网络请求中的常见错误（如服务器错误、超时等），
    提高下载轨道文件的可靠性。
    
    Returns:
        requests.Session: 配置好重试策略的会话对象。
    """
    session = requests.Session()

    # Setup retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )

    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    return session

# Download Precise Orbit
from datetime import datetime, timedelta
orbit_list = _get_orbit_list()
if not orbit_list:
    print("Failed to get orbit file list from ASF server")


def _download_orbit_file_asf(orbit_file: str, output_path: os.PathLike) -> bool:
    url_root = "https://s1qc.asf.alaska.edu/aux_poeorb"
    file_url = f"{url_root}/{orbit_file}"

    try:
        session = _create_session()
        response = session.get(file_url, stream=True, timeout=60)
        response.raise_for_status()

        # Get file size
        total_size = int(response.headers.get('content-length', 0))
        downloaded_size = 0

        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded_size += len(chunk)

                    # Show progress
                    if total_size > 0:
                        progress = (downloaded_size / total_size) * 100
                        print(f"\r  Progress: {progress:.1f}%", end='', flush=True)

        print()  # New line after progress
        return True

    except Exception as e:
        print(f"\n  Download failed: {e}")
        # Clean up partially downloaded file
        if output_path.exists():
            output_path.unlink()
        return False

print(f"Start downloading orbit files from ASF...")
# For each SAFE file, download the corresponding orbit file
for safe_file in safe_files:
    print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
    print(f"Processing: {os.path.basename(safe_file)}")
    
    # Extract mission and timestamp from SAFE 
    basename = os.path.basename(safe_file)

    # Remove .SAFE or .zip extension
    if basename.endswith('.SAFE'):
        basename = basename[:-5]
    elif basename.endswith('.zip'):
        basename = basename[:-4]

    # Split by underscores
    parts = basename.split('_')

    if len(parts) < 6:
        print(f"  Invalid filename format: {basename}")
        continue

    # Mission identifier (S1A or S1B)
    mission = parts[0]

    # Timestamps are typically at positions 5 and 6
    # Format: YYYYMMDDTHHMMSS
    start_time_str = parts[5]
    stop_time_str = parts[6]

    # Parse timestamps
    try:
        start_time = datetime.strptime(start_time_str, '%Y%m%dT%H%M%S')
        stop_time = datetime.strptime(stop_time_str, '%Y%m%dT%H%M%S')
    except ValueError:
        print(f"  Could not parse timestamps from: {basename}")
        continue

    if not mission or not start_time:
        print("  Could not parse SAFE filename, skipping")
        continue

    # Calculate date range for orbit search (one day before and after)
    date1 = (start_time - timedelta(days=1)).strftime('%Y%m%d')
    date2 = (start_time + timedelta(days=1)).strftime('%Y%m%d')

    # Find matching orbit file
    mission_short = mission[-1]
    # Search for matching orbit file
    for orbit_file in orbit_list:
    # Check if file matches mission and contains both dates
        if (f"S1{mission_short}" in orbit_file and
            date1 in orbit_file and date2 in orbit_file):
            # Check if orbit file already exists in orbit directory
            orbit_path = os.path.join(ORBIT_DIR, orbit_file)
            if os.path.exists(orbit_path):
                print(f"  Orbit file already exists: {orbit_file}")
                continue
            # Download orbit file
            success = _download_orbit_file_asf(orbit_file, orbit_path)

            if success:
                print(f"  Successfully downloaded: {orbit_file}")
            else:
                print(f"  Failed to download orbit file for: {os.path.basename(safe_file)}")
    
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished downloading orbit files.")

In [ ]:
from .scripts.update_cslc_config import main as update_cslc_config_main

iargs = ["--safe-dir", RAW_LOCATION, "--orbit-dir", ORBIT_DIR]
